In [ ]:
# notebooks/01_fern_protocol.ipynb

import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Load Physical Telemetry (No Prices) ---
# Assuming standard ERCOT Load Extract
load = pd.read_csv('../data/raw/ercot_load_2026.csv', parse_dates=['Time'])
weather = pd.read_csv('../data/raw/weather_zones_2026.csv', parse_dates=['Time'])

# --- 2. Define Seasonal Baselines ---
# Calculate Q90 Load for previous Januarys (2023-2025)
baseline_q90 = 68000 # Example placeholder MW value derived from historicals

# --- 3. Apply Selection Logic (The "Traceability Rule") ---
# Select intervals strictly based on Load > Baseline
stress_packet = load[load['SystemLoad'] > baseline_q90].copy()

# Add Temperature Trigger (e.g., Temp < 32F in North Zone)
cold_snap = weather[weather['LZ_NORTH_Temp'] < 32]

# Union of triggers
fern_window = pd.concat([stress_packet, cold_snap]).drop_duplicates(subset=['Time']).sort_values('Time')

print(f"Fern Event Packet Defined: {fern_window['Time'].min()} to {fern_window['Time'].max()}")
print("Metric: Price-Blind Selection Complete.")

# --- 4. Visualization (Figure 13.1) ---
plt.figure(figsize=(10,6))
plt.plot(load['Time'], load['SystemLoad'], label='System Load')
plt.axhline(baseline_q90, color='red', linestyle='--', label='Stress Threshold (Q90)')
plt.fill_between(fern_window['Time'], 0, 80000, color='gray', alpha=0.3, label='Fern Packet')
plt.title('Price-Blind Stress Packet Definition')
plt.legend()
plt.show()